In [0]:
!pip install pyonmttok fasttext

In [0]:
!git clone https://github.com/facebookresearch/fastText.git
!cd fastText && mkdir build && cd build && cmake .. && make && make install

Cloning into 'fastText'...
remote: Enumerating objects: 122, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 3653 (delta 53), reused 67 (delta 27), pack-reused 3531
Receiving objects: 100% (3653/3653), 8.16 MiB | 9.63 MiB/s, done.
Resolving deltas: 100% (2278/2278), done.
-- The C compiler identification is GNU 7.4.0
-- The CXX compiler identification is GNU 7.4.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done
-- Generating done
-- Build files have been 

In [0]:
!rm -f en_tg_train.tar.gz
!wget https://www.dropbox.com/s/umd8tyx4wz1wquq/en_tg_train.tar.gz
!rm -f en_tg_train.json
!tar -xzvf en_tg_train.tar.gz
!rm en_tg_train.tar.gz

--2020-01-04 18:55:13--  https://www.dropbox.com/s/erbi4w5fuacyvw9/en_tg_train.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/erbi4w5fuacyvw9/en_tg_train.tar.gz [following]
--2020-01-04 18:55:13--  https://www.dropbox.com/s/raw/erbi4w5fuacyvw9/en_tg_train.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc39dfb4487d544a5c9edec24ae7.dl.dropboxusercontent.com/cd/0/inline/AvhVptwLcoXRwgS-yKn0llcKFoUy1OXrS2QnqozAiqSADO5tCy5rdSxfWHxaU_m71Hhq8h1FQdWJylJB6rzqY5NBjQfabEKeX4c3sDd_Zu2e7WnXalY_UkS-eejQcJSOxV4/file# [following]
--2020-01-04 18:55:13--  https://uc39dfb4487d544a5c9edec24ae7.dl.dropboxusercontent.com/cd/0/inline/AvhVptwLcoXRwgS-yKn0llcKFoUy1OXrS2QnqozAiqSADO5tCy5rdSxfWHxaU_m71Hhq8h1FQdWJylJB6rzqY5NBjQfab

In [0]:
!rm -f en_tg_test.tar.gz
!wget https://www.dropbox.com/s/rw674iic8x5udb3/en_tg_test.tar.gz
!rm -f en_tg_test.json
!tar -xzvf en_tg_test.tar.gz
!rm en_tg_test.tar.gz

--2020-01-04 19:13:59--  https://www.dropbox.com/s/rw674iic8x5udb3/en_tg_test.tar.gz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/rw674iic8x5udb3/en_tg_test.tar.gz [following]
--2020-01-04 19:13:59--  https://www.dropbox.com/s/raw/rw674iic8x5udb3/en_tg_test.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc4f1e7a445c6cdc4960ddafef10.dl.dropboxusercontent.com/cd/0/inline/AvgUfnnryDuafi4-X5mJVYSf1cO1RhqmDVSvY_53d5VC95fuNWjEp3z5gpfn_k57koO7LkMe0PAiRwLSVZqZheqGCIhgrwikdd85OlEuRYVtIyYI3coDFhhBYiu8eb-ezcQ/file# [following]
--2020-01-04 19:14:00--  https://uc4f1e7a445c6cdc4960ddafef10.dl.dropboxusercontent.com/cd/0/inline/AvgUfnnryDuafi4-X5mJVYSf1cO1RhqmDVSvY_53d5VC95fuNWjEp3z5gpfn_k57koO7LkMe0PAiRwLSVZqZheqGCIhgrwik

In [0]:
!wget https://www.dropbox.com/s/7qpfgf8bz77h2ss/en_cat_train_raw_markup.tsv
!wget https://www.dropbox.com/s/bszwshgwbrt328k/en_cat_test_raw_markup.tsv
!head -n 2 en_cat_train_raw_markup.tsv

--2020-01-04 18:15:59--  https://www.dropbox.com/s/7qpfgf8bz77h2ss/en_cat_train_raw_markup.tsv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7qpfgf8bz77h2ss/en_cat_train_raw_markup.tsv [following]
--2020-01-04 18:16:00--  https://www.dropbox.com/s/raw/7qpfgf8bz77h2ss/en_cat_train_raw_markup.tsv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc960ed57a24df81d51b44e1ed5f.dl.dropboxusercontent.com/cd/0/inline/AvgKpOxDkZtKk3tmK95CcgqvtZ0De0XVBz5jbK-uWnikvqbdVxy3PBUu3D0iJ2dV0-o9qqZwh7SH9Lo16QUhNcOqG-Ur3l91gKmAXQAU2YU188_tgeFw6w9IvA4mERH89tA/file# [following]
--2020-01-04 18:16:00--  https://uc960ed57a24df81d51b44e1ed5f.dl.dropboxusercontent.com/cd/0/inline/AvgKpOxDkZtKk3tmK95CcgqvtZ0De0XVBz5jbK-uWnikvqbdVxy3PBUu3D0iJ2dV0-

In [0]:
# https://www.kaggle.com/rmisra/news-category-dataset

!rm -f news-category-dataset.zip
!wget https://www.dropbox.com/s/ua18htwqrkwnfpg/news-category-dataset.zip
!unzip news-category-dataset.zip

--2020-01-04 19:54:58--  https://www.dropbox.com/s/ua18htwqrkwnfpg/news-category-dataset.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ua18htwqrkwnfpg/news-category-dataset.zip [following]
--2020-01-04 19:54:59--  https://www.dropbox.com/s/raw/ua18htwqrkwnfpg/news-category-dataset.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uccd5339f0cc0f16aff82f9fadf5.dl.dropboxusercontent.com/cd/0/inline/Avirip0t2mqYzGn0tLRNjFgEOZ3FYQrKZglCgLU7993NbZo_dI6L3h-AhGSBpgXYFd2GzL0yhLMkv_AXZuiM-VoSfZUSa_lCqjy7UpOIUkIhq7EaXlLnMguDZ7gC_afZ6pw/file# [following]
--2020-01-04 19:54:59--  https://uccd5339f0cc0f16aff82f9fadf5.dl.dropboxusercontent.com/cd/0/inline/Avirip0t2mqYzGn0tLRNjFgEOZ3FYQrKZglCgLU7993NbZo_dI6L3h-AhGSBpgXYFd2GzL0y

In [0]:
import pyonmttok
tokenizer = pyonmttok.Tokenizer("conservative")

def preprocess(text):
    text = str(text).strip().replace("\n", " ").replace("\xa0", " ").lower()
    tokens, _ = tokenizer.tokenize(text)
    text = " ".join(tokens)
    return text

In [0]:
import random

def save_to_ft(records, output_file_name, use_preprocess=True):
    with open(output_file_name, "w") as w:
        random.shuffle(records)
        for d in records:
            title = d["title"] if not use_preprocess else preprocess(d["title"])
            text = d["text"] if not use_preprocess else preprocess(d["text"])
            w.write("__label__{} {} {}\n".format(d["res"], title, text))

In [0]:
import json
import random
from collections import Counter, defaultdict
from sklearn.metrics import cohen_kappa_score

def normalize(text):
    return text.replace("\t", " ").replace("\n", " ").replace('"', '').replace("\xa0", " ")

def convert_to_ft(answers_file_name, original_json, output_file_name, min_votes=3, use_preprocess=True):
    with open(answers_file_name, "r") as r:
        header = tuple(next(r).strip().split("\t"))
        records = []
        for line in r:
            fields = line.strip().split("\t")
            assert len(fields) == len(header), fields
            records.append(dict(zip(header, fields)))

    # Filter honeypots out
    records = [r for r in records if not r["GOLDEN:res"]]

    # Normalize fields
    for r in records:
        r.pop("GOLDEN:res", None)
        r.pop("HINT:text", None)
        for key, value in r.items():
            new_key = key.split(":")[-1]
            r[new_key] = r.pop(key)

    # Restore original urls (to fix a bug)
    with open(original_json, "r") as r:
        data = json.load(r)
        title2url = {normalize(d["title"]): d["url"] for d in data}
        for r in records:
            title = normalize(r["title"])
            if title not in title2url:
                continue
            r["url"] = title2url[title]

    # Calc inter-annotator agreement
    annotator2labels = defaultdict(dict)
    unique_keys = list(set([r["url"] for r in records]))
    unique_workers = list(set([r["worker_id"] for r in records]))
    unique_res = list(set([r["res"] for r in records]))
    res2num = {res: i for i, res in enumerate(unique_res)}
    for r in records:
        annotator2labels[r["worker_id"]][r["url"]] = r["res"]
    worker2labels = {}
    for worker_id in unique_workers:
        worker_labels = []
        worker_res = annotator2labels[worker_id]
        for key in unique_keys:
            if key not in worker_res:
                worker_labels.append(-1)
                continue
            worker_labels.append(res2num[worker_res[key]])
        worker2labels[worker_id] = worker_labels
    scores = []
    for w1, labels1 in worker2labels.items():
        for w2, labels2 in worker2labels.items():
            if w1 == w2:
                continue
            fixed_labels1 = []
            fixed_labels2 = []
            for l1, l2 in zip(labels1, labels2):
                if l1 == -1 or l2 == -1:
                    continue
                fixed_labels1.append(l1)
                fixed_labels2.append(l2)
            if fixed_labels1 and fixed_labels2:
                score = cohen_kappa_score(fixed_labels1, fixed_labels2)
                if -1.0 <= score <= 1.0:
                    scores.append(score)
    print("Avg kappa score: {}".format(sum(scores)/len(scores)))

    results = defaultdict(list)
    for r in records:
        results[r["url"]].append(r["res"])

    data = {r["url"]: r for r in records}
    for url, res in results.items():
        res_count = Counter(res)
        if res_count.most_common(1)[0][1] < min_votes:
            data.pop(url)

    rub_cnt = Counter()
    for _, d in data.items():
        rub_cnt[d["res"]] += 1
    print(rub_cnt.most_common())

    save_to_ft(list(data.values()), output_file_name, use_preprocess)

convert_to_ft("en_cat_train_raw_markup.tsv", "en_tg_train.json", "en_cat_train_markup.txt", min_votes=2, use_preprocess=True)
convert_to_ft("en_cat_test_raw_markup.tsv", "en_tg_test.json", "en_cat_test_markup.txt", min_votes=4, use_preprocess=True)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:576: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)


Avg kappa score: 0.6791098900631102
[('society', 232), ('sports', 107), ('economy', 62), ('entertainment', 57), ('not_news', 44), ('technology', 43), ('other', 24), ('science', 5)]
Avg kappa score: 0.6168435156328119
[('society', 179), ('sports', 62), ('economy', 55), ('entertainment', 48), ('technology', 38), ('not_news', 25), ('science', 14), ('other', 7)]


In [0]:
!cat en_cat_train_markup.txt | wc -l
!cat en_cat_test_markup.txt | wc -l

574
428


In [0]:
import os
import random

def read_news_category_dataset(input_file, output_file, use_preprocess=True):
    assert os.path.exists(input_file)
    records = []
    cat2res = {
        "POLITICS": ("society", 100.0/32739),
        "ENTERTAINMENT": ("entertainment", 100.0/16058),
        "BUSINESS": ("economy", 300.0/5937),
        "CRIME": ("society", 100.0/3405),
        "ARTS & CULTURE": ("entertainment", 100.0/700),
        "CULTURE & ARTS": ("entertainment", 100.0/700),
        "TECH": ("technology", 300.0/2082),
        "SCIENCE": ("science", 300.0/2178),
        "SPORTS": ("sports", 300.0/4884),
        "HEALTHY LIVING": ("not_news", 300.0/6694),
        "THE WORLDPOST": ("society", 100.0/3405),
        "FOOD & DRINK": ("other", 150.0/6226),
        "STYLE & BEAUTY": ("other", 150.0/9649)
    }

    with open(input_file, "r") as r:
        for line in r:
            data = json.loads(line)
            title = data["headline"]
            text = data["short_description"]
            data["title"] = title
            data["text"] = text
            category = data["category"]
            if category in cat2res:
                res, prob = cat2res[category]
                data["res"] = res
                if random.random() < prob:
                    records.append(data)
            # else:
                # print("Skipping: ", category, title)
    
    rub_cnt = Counter()
    for d in records:
        rub_cnt[d["res"]] += 1
    print(rub_cnt.most_common())
    
    save_to_ft(records, output_file, use_preprocess)
    return records

read_news_category_dataset("News_Category_Dataset_v2.json", "nc_markup.txt")
!head nc_markup.txt
!cat nc_markup.txt | wc -l

[('entertainment', 424), ('sports', 328), ('society', 323), ('technology', 315), ('not_news', 311), ('other', 305), ('science', 300), ('economy', 288)]
__label__technology alexa can remember birthdays and other things for you now last week at a conference in france , amazon announced a new alexa feature that makes it possible to use the smart assistant
__label__science this is what earth would be like without the moon humans might never have existed .
__label__other food porn : brilliance from david burke get a taste of chef david burke ' s everyday culinary encounters in his iphone photos of food , from savory peanut brittle to watermelon-and-asparagus salad at david burke kitchen .
__label__technology the fakebook inside facebook this post first appeared at billmoyers.com . beginning in 2004 , mark zuckerberg and his companions made a historic contribution
__label__science maybe colleges should take a lesson from zoos by michael preston , ucf forum columnist what would it take to cre

In [0]:
!wget https://www.dropbox.com/s/no7x1n8acl5ykif/en_vectors_v2.bin

--2020-01-04 19:32:46--  https://www.dropbox.com/s/no7x1n8acl5ykif/en_vectors_v2.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/no7x1n8acl5ykif/en_vectors_v2.bin [following]
--2020-01-04 19:32:47--  https://www.dropbox.com/s/raw/no7x1n8acl5ykif/en_vectors_v2.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc20bf39cb18565e7d1e512c85ad.dl.dropboxusercontent.com/cd/0/inline/AvjWiinW9rml6svSvEYNYFS7jpSp1fs08dqqKlk1rFcWz74zjyEDCtKnSFCwhXsQZ2o0erUsF7JecbQ3HhTw2Fkf1DV4CtPv9JIRkSvaECQtyIDel3xM1ie2-j72tul5NZU/file# [following]
--2020-01-04 19:32:47--  https://uc20bf39cb18565e7d1e512c85ad.dl.dropboxusercontent.com/cd/0/inline/AvjWiinW9rml6svSvEYNYFS7jpSp1fs08dqqKlk1rFcWz74zjyEDCtKnSFCwhXsQZ2o0erUsF7JecbQ3HhTw2Fkf1DV4CtPv

In [0]:
!wget https://raw.githubusercontent.com/facebookresearch/fastText/master/python/doc/examples/bin_to_vec.py
!python bin_to_vec.py en_vectors_v2.bin > en_vectors_v2.vec

--2020-01-04 19:33:02--  https://raw.githubusercontent.com/facebookresearch/fastText/master/python/doc/examples/bin_to_vec.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1120 (1.1K) [text/plain]
Saving to: ‘bin_to_vec.py’

bin_to_vec.py       100%[===================>]   1.09K  --.-KB/s    in 0s      

2020-01-04 19:33:02 (248 MB/s) - ‘bin_to_vec.py’ saved [1120/1120]




In [0]:
!cat en_cat_train_markup.txt > en_cat_train_all.txt
!cat nc_markup.txt >> en_cat_train_all.txt
!shuf en_cat_train_all.txt > en_cat_train_shuf.txt

In [0]:
import random
with open("en_cat_train_shuf.txt", "r") as r, open("en_cat_train_train.txt", "w") as train, open("en_cat_train_val.txt", "w") as val:
    for line in r:
        if random.random() < 0.1:
            val.write(line)
        else:
            train.write(line)
!cat en_cat_train_val.txt | wc -l

341


In [0]:
!fasttext supervised -input en_cat_train_train.txt -pretrainedVectors en_vectors_v2.vec -dim 50 -autotune-validation en_cat_train_val.txt -output en_cat_v2 -autotune-modelsize 10M

Progress: 100.0% Trials:    8 Best score:  0.750733 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  27175
Number of labels: 8
Progress: 100.0% words/sec/thread:  334965 lr:  0.000000 avg.loss:  1.045397 ETA:   0h 0m 0s


In [0]:
!fasttext test en_cat_v2.ftz en_cat_test_markup.txt

N	428
P@1	0.862
R@1	0.862


In [0]:
import fasttext
model = fasttext.load_model("en_cat_v2.ftz")
true_labels = []
predicted_labels = []
errors = []
with open("en_cat_test_markup.txt", "r") as r:
    for line in r:
        words = line.strip().split(" ")
        label = words[0][9:]
        true_labels.append(label)
        text = " ".join(words[1:])
        predicted_label = model.predict([text])[0][0][0][9:]
        if label != predicted_label:
            errors.append((label, predicted_label, text[:100]))
        predicted_labels.append(predicted_label)
for label, predicted_label, text in errors:
    print("T: {} P: {} | {}".format(label, predicted_label, text))

T: technology P: society | coláiste phobal roscrea students experience car accidents in virtual reality tipperary county counci
T: entertainment P: society | could haley ’ s role in trump administration taint a 2024 bid ? columbia , s.c . ( ap ) — with the l
T: society P: not_news | cat food sold nationwide recalled over salmonella concern frozen bags of quest beef cat foot , sold 
T: not_news P: society | ramon nears , northern ilocos norte added to areas under signal no . 2 email address newsletters
T: science P: not_news | studies confirm hpv shot is safe monday , nov . 18 , 2019 ( healthday news ) - - the hpv vaccine giv
T: not_news P: entertainment | from nollywood to netflix : genevieve nnaji ' s rise nigerian star and film director genevieve nnaji
T: not_news P: entertainment | watch vic mensa + his sister cook up a tasty dish in new made from scratch teaser clip chicago rappe
T: science P: not_news | the ways astronauts prep for spaceflight could benefit cancer patients , say r